<a href="https://colab.research.google.com/github/Rhqo/LLM_Study/blob/main/3_chatbot_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install openai -q
!pip install cohere -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.8 MB/s eta 0:00:00


In [89]:
import os
from openai import OpenAI
import cohere
from google.colab import userdata

In [90]:
co = cohere.Client(userdata.get('CO_API_KEY'))
openai_client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)

### ChatGPT Q/A chatbot

In [91]:
SYSTEM_PROMPT = '''You are a helpful Q/A bot that can only reference material from a knowledge base.
All context was pulled from a knowledge base.
If a user asks anything that is not "from the knowledge base," say that you cannot answer.
'''

In [121]:
def test_prompt_openai(prompt, suppress=False, model='gpt-3.5-turbo', **kwargs):
    " a simple function to take in a prompt and run it through a given non-chat model "

    chat_completion = openai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        **kwargs
    )
    answer = chat_completion.choices[0].message.content
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer

In [122]:
def test_prompt_cohere(prompt, suppress=False, model='command-xlarge', **kwargs):
    response = co.generate(
        model=model,
        prompt=prompt,
        **kwargs,
      )
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{response.generations[0].text}')

In [128]:
print("===OpenAI===")
test_prompt_openai('Translate to Korean.\n\nWhere is the nearest restaurant?')
print("\n\n===Cohere===")
test_prompt_cohere('Translate to Korean.\n\nWhere is the nearest restaurant?')

===OpenAI===
PROMPT:
------
Translate to Korean.

Where is the nearest restaurant?
------
RESPONSE
------
가장 가까운 음식점은 어디에 있나요?


===Cohere===
PROMPT:
------
Translate to Korean.

Where is the nearest restaurant?
------
RESPONSE
------
 In which language would you like this query translated? I can translate this query into Korean for you:

매우 가까워서 저희는 가까지 갈 수 있는 주문가가 있습니까? 

This translates to: "Is there a nearby restaurant where we could order?" 

Let me know if this was the language you wanted and if you have any more queries about Korean translation! 


In [129]:
print("===OpenAI===")
test_prompt_openai('Translate to Korean.\n\nenglish:Where is the nearest restaurant?\nkorean:')
print("\n\n===Cohere===")
test_prompt_cohere('Translate to Korean.\n\nenglish:Where is the nearest restaurant?\nkorean:')

===OpenAI===
PROMPT:
------
Translate to Korean.

english:Where is the nearest restaurant?
korean:
------
RESPONSE
------
가장 가까운 식당은 어디에 있나요?


===Cohere===
PROMPT:
------
Translate to Korean.

english:Where is the nearest restaurant?
korean:
------
RESPONSE
------
 저희는 가까이에 식당이 있는지 알 수 있을까요? 
(JuhEE nee go-ga-ee-ee-ssuh shikdang-ee isht-lee-da-ra su iss-UH-suh-guh?) 

Is there a restaurant nearby? 

Let me know if you would like any assistance translating from Korean to English as well! 


In [132]:
class ChatbotGPT():
  def __init__(self, system_prompt, threshold=.8):
    self.conversation = [{'role': 'system', 'content': system_prompt}]
    self.threshold = threshold

  def display_conversation(self):
      '''display the conversation in a pretty format denoting the system, user and assistant differently'''
      for turn in self.conversation:
        role = turn['role']
        content = turn['content']
        if role == 'system':
          print(f'System: {content}')
        elif role == 'user':
          print(f'User: {content}')
        elif role == 'assistant':
          print(f'Assistant: {content}')
        print('------------')

  def user_turn(self, message):
    self.conversation.append({"role": "user", "content": message})
    # best_result = get_best_result_from_pinecone(message)
    # print(best_result)
    # if best_result['score'] >= self.threshold:
    #   # Add to the context to the system prompt
    #   self.conversation[0]['content']+=f'\n\nFrom the knowledge base: "{best_result["text"]}"'

    chatgpt_response = openai_client.chat.completions.create(
      model='gpt-4',
      temperature=0,
      messages=self.conversation
    ).choices[0].message.content.strip()
    self.conversation.append({'role': 'assistant', 'content': chatgpt_response})
    return self.conversation[-1]

In [140]:
c = ChatbotGPT(system_prompt=SYSTEM_PROMPT)

c.user_turn('what are fixed costs?')

{'role': 'assistant',
 'content': 'Fixed costs are business expenses that are not dependent on the level of goods or services produced by the business. They are one of several types of costs a business may incur, and are often time-related, such as salaries or rents being paid per month, and are often contractually agreed upon. Fixed costs are an important aspect of financial analysis for businesses, as they are costs that a business must pay, regardless of its level of output.'}